the purpose of this notebook is to take the big dataframe created in 07.01-baseline_data_merging, and make an aggregate we can use for our baseline "worst case" scenario model. This will then be fed into R, where we will use the MICE package to impute data

changelog:

* 4/16/19: added newagg
* 4/17/19: reformatted the ordering of how code runs, and variable names. added aggregation #3.
* 4/19/19: changed the standardizing so that log(x+1) is now applied prior to standardization. also removed ordinal variables from standardizing algorithm and concat them in later with median 0 and iqr 1 so standardize value is either 0 or 1. values from ordinal are not log transformed.


06/14/19:
# prior to this point my pipeline:
1. first median standardized
2. aggregated
3. converted to 2class
4. train/test split
5. imputed


# a big change will happen in this notebook, I will first:
1. convert to two class (c-/abshort &c+/ablong)
2. split the train and test set
3. median standardize
4. aggregate
5. impute
BEFORE I AGGREGATE and median standardize, as some information may be leaking from the train/test set as i had it previously.

## last run: 6/14/19: sensitivity analysis 3day timewindow

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
from sklearn.model_selection import train_test_split
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.
  # This is added back by InteractiveShellApp.init_path()


In [2]:
#patients of interest from rotation_cohort_generation
from parameters import final_pt_df_v, date, repository_path

#patients of interest from rotation_cohort_generation
final_pt_df2 = final_pt_df_v #pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_final_pt_df2.csv'%(most_updated_patient_df), index_col=0)
del(final_pt_df_v)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]


time: 1.28 s


In [3]:
len(patients)

15412

time: 4.21 ms


In [4]:
save_path= str(repository_path)+'/data/cleaned_merged_agg/'
def save_df(df, df_name='default', save_path=save_path, add_subfolder=False):
    #uses the date and supplied df name and saves to the savepath specified above.
    if df_name == 'default':
        df_name= "%s"%(df)
    
    address=save_path+'%s/'%(folder)
    if not os.path.exists(address):
        print(address)
        os.makedirs(address)
    pd.DataFrame(df).to_csv(Path(address+'%s_%s_cleaned_merged_agg.csv' %(date, df_name)))

time: 7.28 ms


In [5]:
from parameters import lower_window, upper_window, folder, date, time_col, time_var, patient_df


time: 753 µs


In [6]:
#importing cleaned_merged big_df
allFiles = glob.glob(str(repository_path)+ '/data/cleaned_merged/%s/'%(folder) + "{}_*.csv".format(date))

df_list=[]
for element in allFiles:
    df_list.append(element.split('{}_'.format(date))[1].split('_prepped.csv')[0]) #making an list of all my dataframes in order they appear in file

i=0
for name in df_list:
    big_df = pd.read_csv(allFiles[i],  index_col=0)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


time: 12.5 s


In [7]:
#quick housekeeping addition to accomidate older generated data
if len(big_df.loc[big_df['label']=="pao2/fio2",'label'])>1:
    big_df.loc[big_df['label']=="pao2/fio2",'label']="pao2fio2ratio"

time: 374 ms


In [8]:
big_df.head()

,icustay_id,t_0,value,label,uom,delta,source,subject_id
840553,200001.0,2181-11-26,absent,bands,y/n,0 days 00:00:00.000000000,labs,55973
16144,200001.0,2181-11-26,0.0,cancer_elix,y/n,0 days 00:00:00.000000000,cancer_elix,55973
22958,200001.0,2181-11-26,5,daily_sofa,daily_sofa_score,0 days 00:00:00.000000000,sofa,55973
14836,200001.0,2181-11-26,0.0,dobutamine,y/n,0 days 00:00:00.000000000,dobutamine,55973
18928,200001.0,2181-11-26,0.0,dopamine,y/n,0 days 00:00:00.000000000,dopamine,55973


time: 16.7 ms


In [9]:
#overview of all variables and formats
big_df.groupby('label')['value'].describe().sort_values('unique')

,count,unique,top,freq
label,,,,
dobutamine,20982,2,0.0,19345
rrt,19633,2,0.0,17960
nitrite,19633,2,Neg/Not_tested,19095
gender,12904,2,M,7021
epinephrine,21746,2,0.0,19187
dopamine,25211,2,0.0,18518
vasopressin,22874,2,0.0,19117
norepinephrine,42225,2,1.0,26326
leukocyte,19633,2,Neg/Not_tested,16774


time: 2.06 s


## initial data prep
* convert to two class

In [10]:
#convert to two class
final_pt_df2['final_bin'].unique()
two_classes=['C_neg/A_partial','C_pos/A_full']
two_class_icu=final_pt_df2.loc[final_pt_df2.loc[:,"final_bin"].isin(two_classes),['icustay_id','subject_id','final_bin']]


time: 21.5 ms


In [11]:
big_df=big_df.loc[big_df['icustay_id'].isin(list(two_class_icu['icustay_id'])),:].copy()

time: 1.56 s


In [12]:
len(big_df)
big_df['icustay_id'].nunique() 

10305

time: 37.5 ms


### roundabout way of sampling train/test set so that each subject is only in either train or test:
* To ensure a single patient did not end up with samples in both training and testing sets, individual patients (subject_id's) are kept together when performing the stratified train and test split

In [13]:
# label each subject_id with the max of the two classes. 
two_class_pt=two_class_icu.copy()
two_class_pt['final_bin']=pd.factorize(two_class_pt['final_bin'])[0]
two_class_maxsub=two_class_pt.loc[two_class_pt.groupby('subject_id')['final_bin'].idxmax(),:]

time: 1.68 s


In [14]:
# 70/30 train/test set split with 12345=seed, splitting on max final bin of each SUBJECT_ID
train, test = train_test_split(two_class_maxsub, test_size=0.3, random_state=12345, 
                               stratify=two_class_maxsub['final_bin'])
# generate list of each SUBJECT_ID in each split
train_subject=list(train['subject_id'])
test_subject=list(test['subject_id'])

#filtering big_df on train subjects and test subjects to get my train/test splits.
big_df_train= big_df.loc[big_df.loc[:,'subject_id'].isin(train_subject),:].copy()
big_df_test= big_df.loc[big_df.loc[:,'subject_id'].isin(test_subject),:].copy()
del big_df

time: 1.38 s


In [15]:
# #converting venttype to category
# big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']= big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].astype('category')
# #big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].apply(astype('category'))
# big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'] = pd.Categorical(big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'])

time: 861 µs


In [16]:
def category_mapper(big_df):
    
    """
    this function is designed to map some variables currently in categorical format into a ordinal category for the following aggregations. the variables are eventually converted to one-hot-encoding prior to modeling. 
    """
    #converting categories to integers
    mapper={'Mech':2 , 'Oxygen': 1, 'None': 0}
    big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']=big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].replace(mapper).copy()

    #gender_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='gender','value'])[1]
    mapper={'F':0 , 'M': 1}
    big_df.loc[big_df.loc[:,'label']=='gender','value']=big_df.loc[big_df.loc[:,'label']=='gender','value'].replace(mapper).copy()

    #leukocyte_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='leukocyte','value'])[1]
    mapper={'Neg/Not_tested':0 , 'pos': 1}
    big_df.loc[big_df.loc[:,'label']=='leukocyte','value']=big_df.loc[big_df.loc[:,'label']=='leukocyte','value'].replace(mapper).copy()
    #     pd.factorize(big_df.loc[big_df.loc[:,'label']=='leukocyte','value'])[0] 

    #nitrite_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='nitrite','value'])[1]
    mapper={'Neg/Not_tested':0 , 'pos': 1}
    big_df.loc[big_df.loc[:,'label']=='nitrite','value']=big_df.loc[big_df.loc[:,'label']=='nitrite','value'].replace(mapper).copy()
    #     pd.factorize(big_df.loc[big_df.loc[:,'label']=='nitrite','value'])[0] 
    
    # changing pao2/fio2 ratio to a category
    """
    A PaO2/FiO2 ratio less than or equal to 200 is necessary for the diagnosis of acute respiratory distress syndrome by the AECC criteria.[6] 
    The more recent Berlin criteria defines mild ARDS at a ratio of <300.

    A PaO2/FiO2 ratio less than or equal to 250 is one of the minor criteria for severe community acquired pneumonia (i.e., possible indication for inpatient treatment).

    A PaO2/FiO2 ratio less than or equal to 333 is one of the variables in the SMART-COP risk score for intensive respiratory or vasopressor support in community-acquired pneumonia.
    """
    
    pd.to_numeric(big_df[big_df['label']=="pao2fio2ratio"]['value'])
    labels= ['0-200', '201-333', "334-475","476+"]
    bins = pd.IntervalIndex.from_tuples([(0, 200), (200, 333), (333, 475),(475,3000)])

    big_df.loc[big_df['label']=="pao2fio2ratio",'value']=pd.cut(pd.to_numeric(big_df.loc[big_df['label']=="pao2fio2ratio",'value']), bins,right=False, labels=labels)
    big_df.loc[big_df['label']=="pao2fio2ratio",'value'].value_counts()
    return(big_df)


time: 59.6 ms


In [17]:
big_df_train=category_mapper(big_df_train)
big_df_test=category_mapper(big_df_test)

time: 2.99 s


#### qc

In [18]:
big_df_train.loc[big_df_train.loc[:,'label']=='vent_recieved','value'].value_counts()

1    3422
2    3261
0     450
Name: value, dtype: int64

time: 104 ms


In [19]:
big_df_train.loc[big_df_train.loc[:,'label']=='gender','value'].value_counts()

1    2628
0    2147
Name: value, dtype: int64

time: 110 ms


In [20]:
big_df_train.loc[big_df_train.loc[:,'label']=='bands','value'].value_counts()

absent    6254
<10        607
>10        344
Name: value, dtype: int64

time: 113 ms


In [21]:
big_df_train.loc[big_df_train.loc[:,'label']=='pco2','value'].value_counts()

absent    4752
<50       1752
>50        701
Name: value, dtype: int64

time: 111 ms


In [22]:
big_df_train.loc[big_df_train.loc[:,'label']=='gender','value'].value_counts()

1    2628
0    2147
Name: value, dtype: int64

time: 109 ms


In [23]:
big_df_test.loc[big_df_test.loc[:,'label']=='pao2fio2ratio','value'].value_counts()

(475, 3000]    2236
(333, 475]      310
(0, 200]        297
(200, 333]      257
Name: value, dtype: int64

time: 51.5 ms


In [24]:
big_df_train.loc[big_df_train.loc[:,'label']=='pao2fio2ratio','value'].value_counts()

(475, 3000]    5207
(333, 475]      687
(0, 200]        674
(200, 333]      637
Name: value, dtype: int64

time: 113 ms


## calc median/iqr for standardization
take all non-categorical variables for HEALTHY PATIENTS and calculate the median and IQR for them. then will use this to make z scores via:

$$Z=\frac{(X-\widetilde{X}_{-/short})}{(IQR_{-/short})}$$ where $\widetilde{X}_{-/short}$ is the median value of the patients with negative SSC and short duration EAT.


In [25]:
continuous=['daily_sofa',
            'lactate',
            'mingcs',
            'diasbp',
            'heartrate',
            'meanartpress',
            'resprate',
            'sysbp',
            'temperature',
            'hemoglobin',
            'platelet',
            'wbc',
            'calcium',
            'glucose',
            'ph',
            'bicarbonate',
            'bun',
            'chloride',
            'creatinine',
            'inr',
            'potassium',
            'ptt',
            'sodium',
            'bilirubin',
            'spo2',
            'sum_elix']

onetime=['yearsold','height','weight']

vaso_active=['phenylephrine',
            'norepinephrine',
            'vasopressin',
            'dobutamine',
            'dopamine',
            'epinephrine'] 

ordinal=[
            'leukocyte',
            'nitrite',
            'vent_recieved',
            'o2_flow',
            'rrt',
            'pao2fio2ratio',
            'cancer_elix',
            "any_vasoactives",
            'bands', #added 6/13/19
            'pco2' #added 6/13/19
]

categorical=[
            "ethnicity",
            'gender'
]

time: 11.6 ms


In [26]:
def median_label_fxn(big_df):
    global final_pt_df2
    
    healthy_pt=list(final_pt_df2[final_pt_df2['final_bin']=="C_neg/A_partial"]['icustay_id'])

    #filter to only healthy patients filter
    ##splitting big_df, making a copy and restricting it to all values that will be usd in standardizing
    healthy_df=big_df[big_df['label'].isin(continuous+onetime)].copy()
    healthy_df['value']= pd.to_numeric(healthy_df['value']) #converting to numeric 
    healthy_df=healthy_df[healthy_df['icustay_id'].isin(healthy_pt)].copy() #only numerical values for cneg/ab partial pt
    
    #finding the last 24 hours of each healthy patient.
    healthy_pt_end=pd.DataFrame(healthy_df.groupby("icustay_id")['delta'].max())
    healthy_pt_end["start"]=healthy_pt_end['delta']- pd.to_timedelta("1 day 00:00:00")
    healthy_pt_end=healthy_pt_end.rename(columns={"delta":"end"}).reset_index()
    healthy_df= pd.merge(healthy_df,healthy_pt_end, left_on="icustay_id", right_on="icustay_id", how="left" ) #now have the last 24 hours annotated for each patient as start and end.
    
    #calculating medians and iqr for each label based on healthy patient's last 24 hours in icu
    median_label=pd.DataFrame((healthy_df.groupby("label")['value'].median())).reset_index()
    median_label=median_label.rename(columns={'value':"median"})
    iqr_label=pd.DataFrame((healthy_df.groupby("label")['value'].quantile(0.75)-healthy_df.groupby("label")['value'].quantile(0.25))).reset_index()
    iqr_label=iqr_label.rename(columns={'value':"iqr"})
    median_label=pd.merge(median_label,iqr_label)  #final median df
    return(median_label)

def combine_vasoactives(big_noCat, median_label):
    global vaso_active
    ##combining vasoactives
    vaso_active_df=big_noCat[big_noCat['label'].isin(vaso_active)].groupby('icustay_id')['value'].max().reset_index()
    vaso_active_df['uom']="y/n"
    vaso_active_df['label']="any_vasoactives"
    vaso_active_df['delta']=pd.to_timedelta("0days")
    vaso_active_df['source']="any_vasoactives"

    vaso_active_df=pd.merge(vaso_active_df, final_pt_df2[["icustay_id",'subject_id',"t_0"]], how="left", left_on="icustay_id", right_on="icustay_id")
    vaso_active_df=pd.merge(vaso_active_df, median_label, how="left").fillna(0)
    vaso_active_df['standardize']=vaso_active_df['value']
    vaso_active_df['raw_value']=vaso_active_df['value']
    vaso_active_df.head()

    # # #grabing the rest of the variables not suitable for range or mean/std
    big_noCat=pd.concat([big_noCat, vaso_active_df], sort=False)
    return(big_noCat)

def standardization_fxn(big_df):
    global continuous, onetime, vaso_active, ordinal, categorical
    
    median_label=median_label_fxn(big_df)
    
    ### dataformatting: convert all dtypes to a numeric type that pereserves nan. 
    #splitting categorical, ordinal and continuous
    big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:].copy() 

    #continuous and ordinal variables
    big_noCat= big_df.loc[big_df.loc[:,'label'].isin(continuous),:].copy() 
    big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 
    
    ### adding a standardized value (x-median)/iqr  where median is of the last 24 hours in time window for culture neg/ ab partial patients 
    big_noCat=pd.merge(big_noCat, median_label, how="left") 
    #loging values
    big_noCat['median']= np.log(big_noCat['median']+1.0)
    big_noCat['iqr']= np.log(big_noCat['iqr']+1.0)
    big_noCat['raw_value']=big_noCat['value']
    big_noCat['value']=np.log(big_noCat['value']+1.0)
    
    big_noCat['standardize']=((big_noCat['value']-big_noCat['median'])/big_noCat['iqr']).fillna(0) #standardize is log standardized

    #making an ordinal df to concat on
    ord_df=big_df.loc[big_df.loc[:,'label'].isin(ordinal+vaso_active),:].copy()
    ord_df['raw_value']=ord_df['value']
    ord_df['standardize']=ord_df['value']
    ord_df['median']=None
    ord_df['iqr']=None

    big_noCat=pd.concat([big_noCat,ord_df], sort=False)
    
    #making all ordinal values in standardize equal to unstandardized
    big_noCat.loc[big_noCat['label'].isin(ordinal+vaso_active),'standardize']=big_noCat.loc[big_noCat['label'].isin(ordinal+vaso_active),'value']  #do i need to add sparse?
    #ig_noCat.head()
    
    big_noCat['standardize']= big_noCat['standardize'].apply(pd.to_numeric, args=('coerce',)) #errors occuring downstream due to not having numeric, trying this 4/18/19
    
    ##last step, combining vasoactives into 1 feature
    big_noCat=combine_vasoactives(big_noCat, median_label)  

    return(big_noCat, big_categorical)

time: 292 ms


In [27]:
noCat_df_train, cat_df_train = standardization_fxn(big_df_train)
noCat_df_test, cat_df_test = standardization_fxn(big_df_test)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:36: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.


time: 1min 47s


In [28]:
#noCat_df_train[noCat_df_train['label']=='any_vasoactives'].head()#.unique()

time: 601 µs


# aggregation1:
### clincally guided min/max

In [29]:
hi_value= [#'bands',
'bilirubin',
'bun',
'chloride',
'creatinine',
'daily_sofa',
'glucose',
'heartrate',
'inr',
'lactate',
#'pco2',
'potassium',
'ptt',
'resprate',
'temperature',
'weight', 'rrt', 
'phenylephrine', 'norepinephrine', 'vasopressin', 'dobutamine', 'dopamine', 'epinephrine',  #added this and removed individual vasoactive 5/3/19
'yearsold','leukocyte','nitrite','vent_recieved','o2_flow', 
'any_vasoactives', #added this and removed individual vasoactive 5/3/19
'sum_elix', #added 6/4/19
'cancer_elix'  ]#added 6/4/19

low_value=['bicarbonate',
'diasbp',
'hemoglobin',
'meanartpress',
'mingcs',
'ph',
'platelet',
'spo2',
'sysbp']

both_value=['calcium',
'sodium',
'wbc']

#important_ordinal=["any_vasoactives"]
important_onetime=['yearsold','weight','pao2fio2ratio', 'pco2','bands'] #added pco2 and bands here, removed them above 06/14/19


time: 9.66 ms


In [30]:
noCat_df_train['label'].unique()#noCat_df_train['label']=='any_vasoactives'

array(['daily_sofa', 'sum_elix', 'lactate', 'mingcs', 'diasbp',
       'heartrate', 'meanartpress', 'resprate', 'spo2', 'sysbp',
       'temperature', 'hemoglobin', 'platelet', 'wbc', 'calcium',
       'glucose', 'ph', 'bicarbonate', 'bun', 'chloride', 'creatinine',
       'inr', 'potassium', 'ptt', 'sodium', 'bilirubin', 'cancer_elix',
       'dobutamine', 'dopamine', 'epinephrine', 'norepinephrine',
       'o2_flow', 'pao2fio2ratio', 'pco2', 'phenylephrine', 'rrt',
       'vasopressin', 'vent_recieved', 'bands', 'leukocyte', 'nitrite',
       'any_vasoactives'], dtype=object)

time: 108 ms


### running the min/max aggregations.



In [31]:
def merge_cat_agg(num_df, cat_df, ):
    """
    merges the categorical and aggregated dataframes together. 
    """
    
    worst_df=pd.merge(num_df, cat_df, left_on='icustay_id', right_on='icustay_id',how='left')
    worst_df['ethnicity']=worst_df['ethnicity'].astype("category")
    worst_df['gender']=worst_df['gender'].astype("category")
    
    return(worst_df)

def clin_agg(big_noCat, big_cat,big_df, values="standardize"):
    """
    clincally guided aggregations.
    
    values= choose here if want to use standardization or raw values.
    
    note: getting two minor errors, could use some cleaning up at later date.
    """
    global hi_value, low_value, both_value, important_onetime
    #max aggregation for selected variables
    big_max= big_noCat.loc[big_noCat.loc[:,'label'].isin(hi_value),:]
    table = pd.pivot_table(big_max, values=values, columns='label', index=['icustay_id'],aggfunc=max, dropna=False)
    
    #min aggregation for selected variables
    big_min= big_noCat.loc[big_noCat.loc[:,'label'].isin(low_value),:]
    table2 = pd.pivot_table(big_min, values=values, columns='label', index=['icustay_id'],aggfunc=min, dropna=False)
    
    #max&min aggregation for selected variables
    big_both= big_noCat.loc[big_noCat.loc[:,'label'].isin(both_value),:]
    table3 = pd.pivot_table(big_both, values=values, columns='label', index=['icustay_id'],aggfunc=[max,min], dropna=False)
    
    #first left join all different continuous aggregations together. 
    worst_df=pd.merge(table.reset_index(), table2.reset_index(), how='left')
    worst_df=pd.merge(worst_df, table3.reset_index(), left_on='icustay_id', right_on='icustay_id',how='left')
    
    ### formatting categorical to wide format to match the tables/worst_df
    big_cat= big_cat.pivot(
    index='icustay_id',
    values='value',
    columns='label').reset_index() #need to convert to wide format. should be one row per icustay per time. 
    
    ## merging the categorical and aggregated dataframes together. 
    worst_df=merge_cat_agg(worst_df, big_cat) #using max/min aggregates 

    #adding important one_time values to final aggregated
    agg_remaining= big_df.loc[big_df.loc[:,'label'].isin(important_onetime),:]
    agg_table2 = pd.pivot_table(agg_remaining, values='value', columns='label', index=['icustay_id'],aggfunc=[max], dropna=False) 
    agg_table2.columns = agg_table2.columns.get_level_values(1)
    agg_table2=agg_table2.reset_index()
    agg_table2.head()#.rename(columns={})

    worst_df=pd.merge(worst_df, agg_table2, how='left')
    return(worst_df)


time: 78.9 ms


In [32]:
worst_df_train=clin_agg(noCat_df_train, cat_df_train,big_df_train, values="standardize")

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


time: 1min 4s


In [33]:
worst_df_test=clin_agg(noCat_df_test, cat_df_test, big_df_test, values="standardize")

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


time: 28.2 s


In [34]:
worst_df_train.head()

,icustay_id,any_vasoactives,bilirubin,bun,cancer_elix,chloride,creatinine,daily_sofa,dobutamine,dopamine,...,"(min, calcium)","(min, sodium)","(min, wbc)",ethnicity,gender,bands,pao2fio2ratio,pco2,weight,yearsold
0,200012.0,0.0,NaN,NaN,0,NaN,NaN,-0.430677,0,0,...,NaN,NaN,-0.316749,asian,0,>10,"(475, 3000]",absent,51.2,32.9889590780034
1,200014.0,0.0,NaN,0.0407698,0,0.0370891,-0.234465,-0.178747,0,0,...,-0.044882,0.000000,0.012492,unknown/other,1,absent,"(200, 333]",<50,62.0,84.7281861738535
2,200033.0,1.0,NaN,-0.0822292,1,0.00850716,-0.234465,-0.430677,0,0,...,-0.505654,-0.020446,-0.220988,white/nonhispanic,1,absent,"(475, 3000]",absent,74.0,67.1450990252744
3,200036.0,0.0,NaN,0.119501,0,0.020977,0,-0.430677,0,0,...,NaN,0.011917,-0.188701,white/nonhispanic,1,absent,"(475, 3000]",absent,79.0,74.934136467919
4,200055.0,0.0,-0.356425,0.198543,0,-0.0219907,2.30743,0.503859,0,0,...,-0.135281,-0.004030,-0.255600,NaN,NaN,absent,"(475, 3000]",absent,56.0,NaN


time: 25.4 ms


In [35]:
worst_df_train['pao2fio2ratio'].describe()

count            7205
unique              4
top       (475, 3000]
freq             5207
Name: pao2fio2ratio, dtype: object

time: 11.8 ms


In [36]:
list(worst_df_train)

['icustay_id',
 'any_vasoactives',
 'bilirubin',
 'bun',
 'cancer_elix',
 'chloride',
 'creatinine',
 'daily_sofa',
 'dobutamine',
 'dopamine',
 'epinephrine',
 'glucose',
 'heartrate',
 'inr',
 'lactate',
 'leukocyte',
 'nitrite',
 'norepinephrine',
 'o2_flow',
 'phenylephrine',
 'potassium',
 'ptt',
 'resprate',
 'rrt',
 'sum_elix',
 'temperature',
 'vasopressin',
 'vent_recieved',
 'bicarbonate',
 'diasbp',
 'hemoglobin',
 'meanartpress',
 'mingcs',
 'ph',
 'platelet',
 'spo2',
 'sysbp',
 ('max', 'calcium'),
 ('max', 'sodium'),
 ('max', 'wbc'),
 ('min', 'calcium'),
 ('min', 'sodium'),
 ('min', 'wbc'),
 'ethnicity',
 'gender',
 'bands',
 'pao2fio2ratio',
 'pco2',
 'weight',
 'yearsold']

time: 2.46 ms


In [37]:
worst_df_train['vasopressin'].value_counts()

0.0    7070
1.0     135
Name: vasopressin, dtype: int64

time: 4.61 ms


In [38]:
#worst_df_train['vasopressin'].value_counts()

time: 386 µs


In [39]:
worst_df_train['bands'].value_counts()

absent    6254
<10        607
>10        344
Name: bands, dtype: int64

time: 4.08 ms


In [40]:
worst_df_train['pco2'].value_counts()

absent    4752
<50       1752
>50        701
Name: pco2, dtype: int64

time: 4.32 ms


In [41]:
save_df(worst_df_train, 'train')
save_df(worst_df_test, 'test')

/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-opensource/data/cleaned_merged_agg/48_hr_window/
time: 574 ms


In [42]:
del worst_df_train, worst_df_test

time: 6.08 ms


In [43]:
#wd+'/data/processed/merged/{}_worst_df_train_preImp_{}.csv'.format(date,timewindowdays)

time: 466 µs


In [44]:
#print(wd+'/data/processed/merged/{}_worstdf_preImp{}.csv'.format(date,timewindowdays))

time: 411 µs


In [45]:
# class onehot(TransformerMixin):
#     def __init__(self, cols_to_transform):
#         self.cols_to_transform=cols_to_transform
        
#     def transform(self,df ):
#         data = pd.get_dummies(df, columns = self.cols_to_transform, drop_first=True)
#         return(data)
    
#     def fit(self, *_):
#         return self

time: 1.36 ms
